In [2]:
import numpy as np 
import os
import tensorflow as tf
from tensorflow.keras.models import *
from tqdm import tqdm
from random import Random
from model import *

In [3]:
model = getModel(32)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), loss = staticWeightedBincrossentropy, metrics = ['accuracy', tf.keras.metrics.MeanIoU(num_classes = 2)])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 668, 668, 3) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 668, 668, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 666, 666, 32) 896         lambda[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 666, 666, 32) 128         conv2d[0][0]                     
______________________________________________________________________________________________

In [4]:
# paths to preprocessed training data created from trainingData.ipynb
X_PATH = 'E:/SUBDIVS/img'
Y_PATH = 'E:/SUBDIVS/mask'

In [5]:
# used to calculate weight_zero for the staticWeightedBinaryCrossEntropy function
# value = fraction of one pixels / total pixels in training dataset

allY = os.listdir(Y_PATH)

# shuffling data before train/validation split
Random(333).shuffle(allY)

SPLIT_TrainVal = int(len(allY)*0.7)
rollingSum = 0
for num, img in tqdm(enumerate(allY[:SPLIT_TrainVal])):
    tempY = np.load(os.path.join(Y_PATH,img))
    rollingSum += tempY.sum()

print(rollingSum / (SPLIT_TrainVal*484*484))

30998it [02:26, 211.36it/s]

0.03692684935147368


In [ ]:
# trains the model

BATCH_SIZE = 6

allX = os.listdir(X_PATH)
allY = os.listdir(Y_PATH)
allX.sort()
allY.sort()

# seeded shuffling data before train/validation split
Random(333).shuffle(allX)
Random(333).shuffle(allY)

# 70% training, 15% validation, 15% testing
SPLIT_TrainVal = int(len(allX)*0.7)
SPLIT_ValTest = int(len(allX)*0.85)

# iterable function that returns (img, mask) tuples from the training data. Performs augmentation on the data (normal, horizontal flip, vertical flip, horizontal & vertical flip)
def iterTrain():
    for num in range(0, SPLIT_TrainVal*4):
        index = int(num / 4)
        rem = num % 4
        img = np.load(os.path.join(X_PATH,allX[index])).astype(np.uint8)
        mask = np.load(os.path.join(Y_PATH,allY[index])).astype(bool)
        if rem == 0:
            pass
        elif rem == 1:
            img = np.flip(img, axis=0)
            mask = np.flip(mask, axis=0)
        elif rem == 2:
            img = np.flip(img, axis=1)
            mask = np.flip(mask, axis=1)
        else:
            img = np.flip(np.flip(img, axis=0), axis=1)
            mask = np.flip(np.flip(mask, axis=0), axis=1)
        yield img, mask

# iterable function that returns (img, mask) tuples from the validation data
def iterValidate():
    for num in range(SPLIT_TrainVal, SPLIT_ValTest):
        img = np.load(os.path.join(X_PATH,allX[num])).astype(np.uint8)
        mask = np.load(os.path.join(Y_PATH,allY[num])).astype(bool)
        yield img, mask

genTrain = iterTrain()
genValidate = iterValidate()

dataTrain = tf.data.Dataset.from_generator(iterTrain, output_signature=(
    tf.TensorSpec(shape=(668, 668, 3), dtype=tf.uint8),
    tf.TensorSpec(shape=(484, 484, 1), dtype=tf.bool)
)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

dataValidate = tf.data.Dataset.from_generator(iterValidate, output_signature=(
    tf.TensorSpec(shape=(668, 668, 3), dtype=tf.uint8),
    tf.TensorSpec(shape=(484, 484, 1), dtype=tf.bool)
)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# optional
callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='cloudModel', histogram_freq=1, write_images=True, update_freq=200),
        tf.keras.callbacks.ModelCheckpoint('checkpoints/cloudModel_weights.ckpt', verbose=1, save_best_only=True)
        ]

results = model.fit(dataTrain, validation_data=dataValidate, verbose=1, epochs=250, callbacks=callbacks)

# for tensorboard run command in activated python environment: python -m tensorboard.main --logdir="path\to\CloudMask Final\model_name" --host=127.0.0.1

In [3]:
# saves the model
model.load_weights('checkpoints/cloudModel_weights.ckpt/')
model.save('save_cloudModel')

INFO:tensorflow:Assets written to: save_sUNET32V2\assets


In [6]:
# tests the model

allX = os.listdir(X_PATH)
allY = os.listdir(Y_PATH)

model = tf.keras.models.load_model('save_cloudModel', custom_objects={'staticWeightedBincrossentropy': staticWeightedBincrossentropy})

# shuffling data before train/validation split
Random(333).shuffle(allX)
Random(333).shuffle(allY)

SPLIT_TrainVal = int(len(allX)*0.7)
SPLIT_ValTest = int(len(allX)*0.85)

def iterTest():
    for num in range(SPLIT_ValTest, int(len(allX))):
        img = np.load(os.path.join(X_PATH,allX[num])).astype(np.uint8)
        mask = np.load(os.path.join(Y_PATH,allY[num])).astype(bool)
        yield img, mask

genTest = iterTest()

BATCH_SIZE = 1
dataTest = tf.data.Dataset.from_generator(iterTest, output_signature=(
    tf.TensorSpec(shape=(668, 668, 3), dtype=tf.uint8),
    tf.TensorSpec(shape=(484, 484, 1), dtype=tf.bool)
)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

results = model.evaluate(dataTest, verbose=1)
print(results)

6643/6643 [==============================] - 154s 22ms/step - loss: 0.0087 - accuracy: 0.9667 - mean_io_u: 0.4812
[0.008651204407215118, 0.9667173027992249, 0.48119229078292847]
